#Mount to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive

# Clone Sign Language Model

In [ ]:
!git clone https://github.com/RizkaHer/SignLanguageModel.git

#Prepare Prerequisite

All the data below available on github [here.]()
You can put all the data below inside /content/drive/My Drive/SignLanguageModel by creating a shortcut.
1. Get trained dataset [here](https://drive.google.com/drive/folders/16CI9IvBjIH_nmmL_ZyQXugq_oDpB52Vi?usp=sharing)
2. Get model [here](https://drive.google.com/file/d/15AyepiEwXs-VPYuTB7Ji2fKCKCE9v6_x/view?usp=sharing)
3. Get real test data [here](https://drive.google.com/drive/folders/1a5JDsWmY-70FHOLBHFEEYq4hV8p8HqIa?usp=sharing)


# Set Configuration

In [ ]:
%cd /content/drive/My Drive/SignLanguageModel

In [9]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from sign import SignConfig
from sign import SignDataset
import os
import numpy as np

Change the path according to your directory where you save folder "train" and "val"
Inside the folder, there is file name instances_default.json

In [10]:
dataset_train = SignDataset()
dataset_train.load_sign('/content/drive/My Drive/SignLanguageModel/train/instances_default.json', '/content/drive/My Drive/SignLanguageModel/train')
dataset_train.prepare()

dataset_val = SignDataset()
dataset_val.load_sign('/content/drive/My Drive/SignLanguageModel/val/instances_default.json', '/content/drive/My Drive/SignLanguageModel/val')
dataset_val.prepare()

Change "logs" according to your directory where you save .h5

In [18]:
class InferenceConfig(SignConfig): 
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
ROOT_DIR = os.path.abspath("../content/drive/My Drive")
inference_config = InferenceConfig()
DETECTION_MIN_CONFIDENCE = 0.9
MODEL_DIR = os.path.join("../content/drive/My Drive/SignLanguageModel/")

Change the path according to your directory where you save .h5



In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

model_path = "mask_rcnn_sign.h5"
model.load_weights(model_path, by_name=True)
print("Loading weights from ", model_path)

Change real_test_dir to the path where you save the image for testing

In [20]:
import skimage
real_test_dir = '/content/drive/My Drive/SignLanguageModel/real_dataset'
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    print( r['class_ids'])
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                dataset_val.class_names, r['scores'], figsize=(5,5))
    

KeyboardInterrupt: ignored